# Imports & load dataset

In [1]:
%load_ext autoreload
%autoreload 2

from torch.nn.utils import prune
import zipfile
from airfoil_diffusion.airfoil_datasets import *
from airfoil_diffusion.networks import *
from airfoil_diffusion.trainer import *

In [2]:
if not os.path.exists("./datasets/1_parameter/data/"):
    files=[file for file in os.listdir("./datasets/1_parameter/") if file.endswith(".zip")]
    for file in tqdm(files): 
        f=zipfile.ZipFile("./datasets/1_parameter/"+file,'r')
        for file in f.namelist():
            f.extract(file,"./datasets/1_parameter/data/")
        f.close() 

train_dataset = AirfoilDataset(FileDataFiles("./datasets/1_parameter/train_cases.txt",base_path="./datasets/1_parameter/data/"),
                               data_size=32)

Loading data:   0%|          | 0/125 [00:00<?, ?it/s]

Loading data: 100%|██████████| 125/125 [00:05<00:00, 24.45it/s]


# Proves

In [3]:
network_configs = {
    # "attention_layers": [2, 3],
    # "condition_layers": [-2],
    # "depth_each_layer": 2,
    "dim_basic": 16,
    # "dim_condition": 3,
    # "dim_encoded_time": 8,
    "dim_in": 3,
    # "dim_multipliers": [1, 2, 4, 4],
    # "dim_multipliers": [2, 2, 2, 2],
    "dim_multipliers": [2,2],
    "dim_out": 3,
    # "heads_attention": 4,
    # "linear_attention": False,
    "skip_connection_scale": 0.707,
    # "use_input_condition": True
}
model_weights = f"./training/single_parameter/32/training/2024-08-29-13_18_52/checkpoints/checkpoint_125000.pt"
model = AifNet(**network_configs)
# model = UNet(**network_configs)
# model.load_state_dict(torch.load(model_weights)["network"])

In [4]:
model.show_current_configs()

dim_basic: 16
dim_in: 6
dim_multipliers: [2, 2]
dim_out: 3
skip_connection_scale: 0.707
attention_layers: [3, 4]
condition_layers: [-2]
use_input_condition: True
depth_each_layer: 2
dim_encoded_time: 8
dim_condition: 3
heads_attention: 4
linear_attention: False
condition_dim: 0


# Train model

In [15]:
network_configs = {
    "attention_layers": [2, 3],
    "condition_layers": [-2],
    "depth_each_layer": 2,
    "dim_basic": 16,
    "dim_condition": 3,
    "dim_encoded_time": 8,
    "dim_in": 3,
    # "dim_multipliers": [1, 2, 4, 4],
    # "dim_multipliers": [2, 2, 2, 2],
    # "dim_multipliers": [2, 2, 2],
    "dim_multipliers": [2,2],
    # "dim_multipliers": [1,1],
    "dim_out": 3,
    "heads_attention": 4,
    "linear_attention": False,
    "skip_connection_scale": 0.707,
    "use_input_condition": True
}

In [16]:
network = AifNet(**network_configs)
network.show_current_configs()

attention_layers: [2, 3]
condition_layers: [-2]
depth_each_layer: 2
dim_basic: 16
dim_condition: 3
dim_encoded_time: 8
dim_in: 6
dim_multipliers: [2, 2]
dim_out: 3
heads_attention: 4
linear_attention: False
skip_connection_scale: 0.707
use_input_condition: True
condition_dim: 0


In [7]:
diffusion_trainer = DiffusionTrainer()
train_configs = {
    "name": "training",
    "save_path": "./training/single_parameter/32/",
    "device": "cuda:0",
    "batch_size_train": len(train_dataset),
    "shuffle_train": True,
    "num_workers_train": 0,
    "validation_epoch_frequency": 0,
    "optimizer": "AdamW",
    "lr_scheduler": "step",
    "warmup_epoch": 0,
    "record_iteration_loss": False,
    "epochs": 125000,
    "save_epoch": 5000,
    "lr": 0.0001,
    "final_lr": 0.00001
}

In [ ]:
diffusion_trainer.train_from_scratch(network, train_dataset, **train_configs)

In [8]:
def evaluate_sparcity(model):
    total_params = 0
    total_zeros = 0

    for name, module in model.named_modules():
        if hasattr(module, "weight"):
            num_params = sum(p.numel() for p in module.parameters())
            total_params += num_params
            total_zeros += torch.sum(module.weight == 0)

    return total_params, total_zeros.item()


In [9]:
def _prune(network: nn.Module, prune_type:str, pruning_percentage:float):
    norm_n = {'L1': 1, 'L2': 2}[prune_type]

    for module in network.modules():
        if type(module) is nn.Conv2d and module.out_channels > 3:
            prune.ln_structured(module, 'weight', amount=pruning_percentage, dim=0, n=norm_n)


def prune_remove(network):
    for module in network.modules():
        if type(module) is nn.Conv2d and module.out_channels > 3:
            prune.remove(module, 'weight')


In [13]:
_prune(network, "L1", 0.5)

In [14]:
total_w, n_zeros = evaluate_sparcity(network)
print(f"Sparse: {100 * n_zeros / total_w:.2f}%")
print(f"Non zeros: {total_w - n_zeros}")

Sparse: 0.00%
Non zeros: 43794


In [87]:
0.92**8

0.5132188731375618

In [39]:
network = AifNet(**network_configs)
for _ in range(17):
    _prune(network, "L1", (1 - 0.9))

In [40]:
total_w, n_zeros = evaluate_sparcity(network)
print(f"Sparse: {100 * n_zeros / total_w:.2f}%")
print(f"Non zeros: {total_w - n_zeros}")

Sparse: 73.49%
Non zeros: 41265


In [16]:
prune_remove(network)

In [ ]:
# 117618